# Import Count Matrix and Filter Genes/Cells

In [ ]:
from utilities import *

# Load your count_matrix DataFrame (this code assumes that user has already performed scTransform or their preferred normalization method)
count_matrix = pd.read_csv("path to your count matrix", index_col=0)

# Filter cells where at least 5% of genes are expressed
count_matrix = count_matrix.loc[:, (count_matrix > 0).sum(axis=0) > 0.05 * count_matrix.shape[0]]

# Filter genes that are expressed in more than 5% of the cells
count_filtered = count_matrix.loc[(count_matrix > 0).sum(axis=1) > 0.05 * count_matrix.shape[1], :]

row_means = np.mean(count_filtered, axis=1)

# Check if any row means are 0 or 1
rows_with_mean_zero_or_one = (row_means == 0) | (row_means == 1)

# If you need the indices of such rows
indices = np.where(rows_with_mean_zero_or_one)[0]

row_means = np.mean(count_filtered, axis=1)

# Identify the indices of rows with mean 0 or 1
rows_to_remove = (row_means == 0) | (row_means == 1)

# Remove these rows
count_filtered = count_filtered[~rows_to_remove]

# Create coexpression network

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

df = pd.DataFrame(count_filtered)
so = create_sifinet_object(df, rowfeature= True)
so = quantile_thres2(so)
so = cal_coexp(so, X = so.data_thres['dt'], X_full = so.data_thres['dt'])
so = create_network(so, alpha=0.05, manual=False, least_edge_prop=0.01)
so = filter_lowexp(so, t1=10, t2=0.9, t3=0.9)

In [ ]:
# This code takes the coexpression matrix and assigns edges to our network using the previously obtained threshold, "sif_ob_test.thres"

sif_ob_test = so

# Perform the element-wise comparison and assignment
adj_matrix = pd.DataFrame(np.where(
    np.abs(sif_ob_test.coexp - sif_ob_test.est_ms['mean']) > sif_ob_test.thres,
    np.abs(sif_ob_test.coexp),
    0
))

adj_matrix.index = df.index
adj_matrix.columns = df.index
adj_matrix = convert_gene_ids(adj_matrix, "ensembl")

# This converts ensemble (or symbol ids) into the appropriate entrez ids
adj_matrix = convert_gene_ids(adj_matrix, "ensembl")

G_s = create_network_from_adj_matrix(adj_matrix)

# This retrieves pathway information from the KEGG database
categorized_pathways = get_categorized_pathways()

#  Specify pathways of interest (list of pathways is stored in categorized_pathways) and run CPACT with the "analyze_pathways" function

In [ ]:
test_pathways = ["Cellular senescence", "Cell cycle", "Apoptosis"]
pathway_results = analyze_pathways(G_s, test_pathways, categorized_pathways ,num_walks=200, max_walk_length = 200)